In [1]:
import networkx as nx

from pathlib import Path
from dreaminsg_integrated_model.src.network_recovery import *
import dreaminsg_integrated_model.src.simulation as simulation
from dreaminsg_integrated_model.src.network_sim_models.integrated_network import *

from dreaminsg_integrated_model.src.network_sim_models.interdependencies import *

from dreaminsg_integrated_model.src.optimizer import *
import dreaminsg_integrated_model.src.plots as model_plots

import random

In [2]:
micropolis_network = IntegratedNetwork(name = 'Micropolis')

In [3]:
MAIN_DIR = Path('../..')
network_dir= 'micropolis'
water_file = MAIN_DIR/f'dreaminsg_integrated_model/data/networks/{network_dir}/water/water.inp'
power_file = MAIN_DIR/f'dreaminsg_integrated_model/data/networks/{network_dir}/power/power.json'
transp_folder = MAIN_DIR/f'dreaminsg_integrated_model/data/networks/{network_dir}/transportation/'

# load all infrastructure networks
micropolis_network.load_networks(water_file, power_file, transp_folder, power_sim_type = '3ph')

C:\Users\srijith\anaconda3\envs\redcar\lib\site-packages\wntr\epanet\io.py:2173: UserWarning: Not all curves were used in "..\..\dreaminsg_integrated_model\data\networks\micropolis\water\water.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


Water network successfully loaded from ..\..\dreaminsg_integrated_model\data\networks\micropolis\water\water.inp. The analysis type is set to Pressure Dependent Demand Analysis.
initial simulation duration: 60s; hydraulic time step: 60s; pattern time step: 3600s

Power system successfully loaded from ..\..\dreaminsg_integrated_model\data\networks\micropolis\power\power.json. Three phase power flow simulation will be used.

Transportation network successfully loaded from ..\..\dreaminsg_integrated_model\data\networks\micropolis\transportation. Static traffic assignment method will be used to calculate travel times.


In [4]:
pn = micropolis_network.pn

In [5]:
G_power = nx.Graph()

In [6]:
power_nodes = pd.DataFrame(
    columns=["id", "node_type", "node_category", "x", "y"]
)

for index, row in pn.bus.iterrows():
    power_nodes = power_nodes.append(
        {
            "id": row["name"],
            "node_type": "power_node",
            "node_category": "Bus",
            "x": pn.bus_geodata.x[index],
            "y": pn.bus_geodata.y[index],
        },
        ignore_index=True,
    )

In [7]:
power_links = pd.DataFrame(
    columns=["id", "link_type", "link_category", "from", "to"]
)

for _, row in pn.line.iterrows():
    power_links = power_links.append(
        {
            "id": row["name"],
            "link_type": "Power",
            "link_category": "Power line",
            "from": pn.bus.name.values[row["from_bus"]],
            "to": pn.bus.name.values[row["to_bus"]],
        },
        ignore_index=True,
    )

for _, row in pn.trafo.iterrows():
    power_links = power_links.append(
        {
            "id": row["name"],
            "link_type": "Power",
            "link_category": "Transformer",
            "from": pn.bus.name.values[row["hv_bus"]],
            "to": pn.bus.name.values[row["lv_bus"]],
        },
        ignore_index=True,
    )

for _, row in pn.switch[pn.switch.et == "b"].iterrows():
    power_links = power_links.append(
        {
            "id": row["name"],
            "link_type": "Power",
            "link_category": "Switch",
            "from": pn.bus.name.values[row["bus"]],
            "to": pn.bus.name.values[row["element"]],
        },
        ignore_index=True,
    )

In [8]:
G_power = nx.from_pandas_edgelist(
            power_links,
            source="from",
            target="to",
            edge_attr=True,
        )

In [9]:
for index, row in power_nodes.iterrows():
    G_power.nodes[row["id"]]["node_type"] = row["node_type"]
    G_power.nodes[row["id"]]["node_category"] = row["node_category"]
    G_power.nodes[row["id"]]["coord"] = (row["x"], row["y"])

In [10]:
pos = {node: G_power.nodes[node]["coord"] for node in power_nodes.id}
model_plots.plot_bokeh_from_integrated_graph(G_power, title = "Micropolis power")

Loading BokehJS ...